In [1]:
#update the metadata info
import scanpy as sc #for scanpy >= 1.3.7
import anndata as ann
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import logging
import os
import glob
import matplotlib
import math

In [2]:
path = '/domino/edv/id-td-virology/Zhiyuan/public/Gut_remap/processed_data/pp_2.h5ad'
adata_raw = sc.read_h5ad(path)
savepath = '/domino/edv/id-td-virology/Zhiyuan/public/Gut_remap/processed_data/B_cells.h5ad'
adata_B = sc.read_h5ad(savepath)

In [25]:
#nadia_csv
meta_path = '/domino/edv/id-td-virology/Public_dataset/2022_Gut/aggr/analysis/all_annotated_merge/20220825_seurat_Gut_annotated.csv'
meta_nadia = pd.read_csv(meta_path, index_col = 0)


In [18]:
meta_nadia

,sample_id,n_genes_by_counts,total_counts,pct_counts_in_top_50_genes,total_counts_mt,pct_counts_mt,cell_quality,nCount_RNA,nFeature_RNA,RNA_snn_res.0.1,seurat_clusters,FinalAnnotation,SecondAnnotation,GeneralAnnotation
AAACCTGAGACAGACC-1-P190801_Liver_1,P190801_Liver_1,1550,3978,31.196581,265,6.661639,high,3978,1550,0,0,CD8+Texhausted,CD8+,CD3+
AAACCTGAGAGAGCTC-1-P190801_Liver_1,P190801_Liver_1,1482,3196,30.444305,300,9.386733,high,3196,1482,0,0,MAIT,CD8+,CD3+
AAACCTGAGCCTCGTG-1-P190801_Liver_1,P190801_Liver_1,917,1997,32.899349,116,5.808713,high,1997,917,0,0,CD8+TEffector,CD8+,CD3+
AAACCTGAGCGATATA-1-P190801_Liver_1,P190801_Liver_1,522,743,30.013459,50,6.729475,low,743,522,4,4,NK,NK,NK
AAACCTGAGTGACTCT-1-P190801_Liver_1,P190801_Liver_1,3216,15616,34.976947,825,5.283043,high,15616,3216,8,8,Doublets,Doublets,Doublets
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAGTTACGGAG-1-P190402_Liver_1,P190402_Liver_1,1777,4700,30.723404,125,2.659575,high,4700,1777,0,0,CD4+TResidentMemory,CD4+,CD3+
TTTGTCAGTTCAGACT-1-P190402_Liver_1,P190402_Liver_1,1775,4706,26.285593,170,3.612410,high,4706,1775,0,0,CD8+TEffector,CD8+,CD3+
TTTGTCATCAGGCCCA-1-P190402_Liver_1,P190402_Liver_1,1598,3651,25.965489,197,5.395782,high,3651,1598,0,0,CD8+TEffector,CD8+,CD3+
TTTGTCATCGCGTTTC-1-P190402_Liver_1,P190402_Liver_1,1398,2757,26.187885,150,5.440696,high,2757,1398,0,0,CD8+TEffector,CD8+,CD3+


In [8]:
adata_raw[meta_nadia.index,:].obs.groupby('cell_quality').size()

cell_quality
high    288764
dtype: int64

In [9]:
adata_raw.obs.groupby('cell_quality').size()

cell_quality
high    288784
low       4776
dtype: int64

In [11]:
#make the small changes to set some edges cells to low-quliaty, to be consistant with nadia
ind = [x for x in adata_raw.obs.loc[adata_raw.obs.cell_quality == 'high'].index if x not in meta_nadia.index]

In [16]:
adata_raw.obs.loc[ind,'cell_quality'] = 'low'

In [17]:
adata_raw[meta_nadia.index,:].obs.groupby('cell_quality').size()

cell_quality
high    288764
dtype: int64

In [41]:
#start modify nadia's metadata, adding B cell annotation
ind_mB = adata_B.obs.loc[adata_B.obs.subtype == 'Memory B'].index
ind_nB = adata_B.obs.loc[adata_B.obs.subtype == 'Naive B'].index
ind_asB = adata_B.obs.loc[adata_B.obs.subtype == 'Antigen stimulated B'].index
ind_p = adata_B.obs.loc[adata_B.obs.subtype == 'Plasma'].index
ind_d = adata_B.obs.loc[adata_B.obs.subtype == 'doublets'].index

In [42]:
meta_nadia.loc[ind_mB, 'FinalAnnotation'] = 'Memory B'
meta_nadia.loc[ind_nB, 'FinalAnnotation'] = 'Naive B'
meta_nadia.loc[ind_asB, 'FinalAnnotation'] = 'Activated B'
meta_nadia.loc[ind_p, 'FinalAnnotation'] = 'Plasma'
meta_nadia.loc[ind_d, 'FinalAnnotation'] = 'Doublets'

In [43]:
#transfer cluster 12 in leidenr0.8 from Plasma to doublets
ind_P_d = adata_B.obs.loc[adata_B.obs['leiden_r0.8'] == '12'].index
meta_nadia.loc[ind_P_d, 'FinalAnnotation'] = 'Doublets'

In [44]:
#synchronize the changes to SecondAnnotation and GeneralAnnotation
meta_nadia.loc[meta_nadia.FinalAnnotation == 'Doublets', 'SecondAnnotation'] = 'Doublets'
meta_nadia.loc[meta_nadia.FinalAnnotation == 'Doublets', 'GeneralAnnotation'] = 'Doublets'

meta_nadia.loc[meta_nadia.FinalAnnotation == 'Plasma', 'SecondAnnotation'] = 'Plasma'
meta_nadia.loc[meta_nadia.FinalAnnotation == 'Plasma', 'GeneralAnnotation'] = 'Plasma'

In [47]:
meta_nadia.groupby('FinalAnnotation').size()

FinalAnnotation
Activated B               2472
CD4+CentralMemory        27239
CD4+Naive                20593
CD4+TNaive                5073
CD4+TResidentMemory       6468
CD4+Th1-like              2510
CD4+Treg                  2635
CD8+EffectorMemory        2330
CD8+Naive                 9262
CD8+TCentralMemory        6510
CD8+TEffector            36617
CD8+Texhausted           25507
DN-Tcell                   529
Doublets                 15206
KupfferCell                178
LSEC                       653
MAIT                     16586
Macrophage                 992
Memory B                  9673
MonocytesClassical       20409
MonocytesNonClassical     3400
NK                       33361
NK-Resident              12688
Naive B                  12362
Plasma                    1627
Platelets                 1833
Proliferating             2365
cDC1                      1212
cDC2                      1544
gdT                       6283
pDC                        647
dtype: int64

In [49]:
#now adding patient_info
path_1 = '/domino/edv/id-td-virology/Zhiyuan/public/Gut_remap/processed_data/patient_info.csv'
p_info_1 = pd.read_csv(path_1, index_col = 0)

path_2 = '/domino/edv/id-td-virology/Zhiyuan/public/Gut_remap/processed_data/patient_info_2.csv'
p_info_2 = pd.read_csv(path_2, index_col = 0)

In [50]:
p_info_1 

,Gender,Disease Stage
Individual Name,,
P190326,male,IT
P190402,male,IT
P190604,male,IT
P190716,male,AR
P190719,male,IA
Dhc570,male,HC
D528848,male,HC
D529074,female,HC
D529351,male,HC


In [51]:
p_info_2

,Age (years),Time since HBV diagnosis (years),HBsAg (COI),HBsAb (IU/L),HBeAg (COI),HBeAb (COI),HBcAb (COI),HBsAg level (IU/ml),HBV DNA level (log10 IU/ml),ALT (U/L),AST (U/L),Stage
Patient ID,,,,,,,,,,,,
P190326,40s,27,+,-,+,-,+,5652,8.21,43.0,32.0,G1-2S2
P190402,20s,1.5,+,-,+,-,+,47779,8.28,46.0,24.0,G0-1S1
P190604,30s,20,+,-,+,-,+,44441,8.6,38.0,42.0,G1S1
P190808,50s,20,+,-,+,-,+,69050,8.88,30.0,26.0,G1S1
P190902,30s,5,+,-,+,-,+,36830,＞8,20.0,19.0,G1S1
P190910,20s,NaN,+,-,+,-,+,29031,＞8,40.0,29.0,G1S1
P190719,20s,NaN,+,-,+,-,+,32970,＞8,262.0,91.0,G3S4
P190801,20s,> 20,+,+,+,-,+,2527,7.01,224.0,91.0,G2S3
P190911,40s,20,+,-,+,-,+,1218,7.22,191.0,111.0,G3S4


In [53]:
meta_nadia['patient_id'] = meta_nadia.sample_id.str.split('_', expand = True)[0]
meta_nadia['tissue'] = meta_nadia.sample_id.str.split('_', expand = True)[1]

In [55]:
meta_nadia.groupby('tissue').size()

tissue
Blood    164679
Liver    124085
dtype: int64

In [57]:
meta_nadia.groupby('patient_id').size()

patient_id
D528848    13846
D529074    14332
D529351     6842
D529354     8067
D529409     6345
Dhc570      4432
P190326     5076
P190402     7725
P190604     7470
P190716     8787
P190719     7942
P190801     7063
P190808    14016
P190902    11381
P190910    12172
P190911    19057
P191008    23649
P191028    20603
P191112    18926
P191126    12955
P191127    21423
P191210    18392
P191217    18263
dtype: int64

In [58]:
df = meta_nadia.copy()

In [61]:
p_info_1.columns = ['Gender', 'Group']

In [63]:
for ind in p_info_1.index:
    for c_name in p_info_1.columns:
        item = p_info_1.loc[ind, c_name]
        c_id = df.loc[df.patient_id == ind].index
        df.loc[c_id, c_name] = item

In [68]:
p_info_2.fillna('N.A.', inplace = True)

In [69]:
p_info_2

,Age (years),Time since HBV diagnosis (years),HBsAg (COI),HBsAb (IU/L),HBeAg (COI),HBeAb (COI),HBcAb (COI),HBsAg level (IU/ml),HBV DNA level (log10 IU/ml),ALT (U/L),AST (U/L),Stage
Patient ID,,,,,,,,,,,,
P190326,40s,27,+,-,+,-,+,5652,8.21,43.0,32.0,G1-2S2
P190402,20s,1.5,+,-,+,-,+,47779,8.28,46.0,24.0,G0-1S1
P190604,30s,20,+,-,+,-,+,44441,8.6,38.0,42.0,G1S1
P190808,50s,20,+,-,+,-,+,69050,8.88,30.0,26.0,G1S1
P190902,30s,5,+,-,+,-,+,36830,＞8,20.0,19.0,G1S1
P190910,20s,N.A.,+,-,+,-,+,29031,＞8,40.0,29.0,G1S1
P190719,20s,N.A.,+,-,+,-,+,32970,＞8,262.0,91.0,G3S4
P190801,20s,> 20,+,+,+,-,+,2527,7.01,224.0,91.0,G2S3
P190911,40s,20,+,-,+,-,+,1218,7.22,191.0,111.0,G3S4


In [70]:
for ind in p_info_2.index:
    for c_name in p_info_2.columns:
        item = p_info_2.loc[ind, c_name]
        c_id = df.loc[df.patient_id == ind].index
        df.loc[c_id, c_name] = item

In [78]:
df.groupby('FinalAnnotation').size()

FinalAnnotation
Activated B               2472
CD4+CentralMemory        27239
CD4+Naive                20593
CD4+TNaive                5073
CD4+TResidentMemory       6468
CD4+Th1-like              2510
CD4+Treg                  2635
CD8+EffectorMemory        2330
CD8+Naive                 9262
CD8+TCentralMemory        6510
CD8+TEffector            36617
CD8+Texhausted           25507
DN-Tcell                   529
Doublets                 15206
KupfferCell                178
LSEC                       653
MAIT                     16586
Macrophage                 992
Memory B                  9673
MonocytesClassical       20409
MonocytesNonClassical     3400
NK                       33361
NK-Resident              12688
Naive B                  12362
Plasma                    1627
Platelets                 1833
Proliferating             2365
cDC1                      1212
cDC2                      1544
gdT                       6283
pDC                        647
dtype: int64

In [79]:
df

,sample_id,n_genes_by_counts,total_counts,pct_counts_in_top_50_genes,total_counts_mt,pct_counts_mt,cell_quality,nCount_RNA,nFeature_RNA,RNA_snn_res.0.1,...,HBsAg (COI),HBsAb (IU/L),HBeAg (COI),HBeAb (COI),HBcAb (COI),HBsAg level (IU/ml),HBV DNA level (log10 IU/ml),ALT (U/L),AST (U/L),Stage
AAACCTGAGACAGACC-1-P190801_Liver_1,P190801_Liver_1,1550,3978,31.196581,265,6.661639,high,3978,1550,0,...,+,+,+,-,+,2527,7.01,224.0,91.0,G2S3
AAACCTGAGAGAGCTC-1-P190801_Liver_1,P190801_Liver_1,1482,3196,30.444305,300,9.386733,high,3196,1482,0,...,+,+,+,-,+,2527,7.01,224.0,91.0,G2S3
AAACCTGAGCCTCGTG-1-P190801_Liver_1,P190801_Liver_1,917,1997,32.899349,116,5.808713,high,1997,917,0,...,+,+,+,-,+,2527,7.01,224.0,91.0,G2S3
AAACCTGAGCGATATA-1-P190801_Liver_1,P190801_Liver_1,522,743,30.013459,50,6.729475,low,743,522,4,...,+,+,+,-,+,2527,7.01,224.0,91.0,G2S3
AAACCTGAGTGACTCT-1-P190801_Liver_1,P190801_Liver_1,3216,15616,34.976947,825,5.283043,high,15616,3216,8,...,+,+,+,-,+,2527,7.01,224.0,91.0,G2S3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAGTTACGGAG-1-P190402_Liver_1,P190402_Liver_1,1777,4700,30.723404,125,2.659575,high,4700,1777,0,...,+,-,+,-,+,47779,8.28,46.0,24.0,G0-1S1
TTTGTCAGTTCAGACT-1-P190402_Liver_1,P190402_Liver_1,1775,4706,26.285593,170,3.612410,high,4706,1775,0,...,+,-,+,-,+,47779,8.28,46.0,24.0,G0-1S1
TTTGTCATCAGGCCCA-1-P190402_Liver_1,P190402_Liver_1,1598,3651,25.965489,197,5.395782,high,3651,1598,0,...,+,-,+,-,+,47779,8.28,46.0,24.0,G0-1S1
TTTGTCATCGCGTTTC-1-P190402_Liver_1,P190402_Liver_1,1398,2757,26.187885,150,5.440696,high,2757,1398,0,...,+,-,+,-,+,47779,8.28,46.0,24.0,G0-1S1


In [98]:
#uniforming the datatype to str
df['ALT (U/L)'] = df['ALT (U/L)'].astype('str')
df['AST (U/L)'] = df['AST (U/L)'].astype('str')

In [99]:
save_meta_path = '/domino/edv/id-td-virology/Public_dataset/2022_Gut/aggr/analysis/all_annotated_merge/20220825_seurat_Gut_annotated_Zhiyuan.csv'
df.to_csv(save_meta_path)






In [82]:
#update the raw data
temp = adata_raw.copy()

In [83]:
temp

AnnData object with n_obs × n_vars = 293560 × 36601
    obs: 'sample_id', 'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_50_genes', 'total_counts_mt', 'pct_counts_mt', 'cell_quality'
    var: 'gene_ids', 'feature_types', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [84]:
#add annotation info
columns_to_add = ['FinalAnnotation','SecondAnnotation', 'GeneralAnnotation']
for c in columns_to_add:
    temp.obs[c] = 'unknown'
    temp.obs.loc[df.index, c] = df.loc[:,c]
    

In [87]:
#add patient info
temp.obs['patient_id'] = temp.obs.sample_id.str.split('_', expand = True)[0]
temp.obs['tissue'] = temp.obs.sample_id.str.split('_', expand = True)[1]

In [88]:
for ind in p_info_1.index:
    for c_name in p_info_1.columns:
        item = p_info_1.loc[ind, c_name]
        c_id = temp.obs.loc[temp.obs.patient_id == ind].index
        temp.obs.loc[c_id, c_name] = item

In [89]:
for ind in p_info_2.index:
    for c_name in p_info_2.columns:
        item = p_info_2.loc[ind, c_name]
        c_id = temp.obs.loc[temp.obs.patient_id == ind].index
        temp.obs.loc[c_id, c_name] = item

In [96]:
#uniforming the datatype to str
temp.obs['ALT (U/L)'] = temp.obs['ALT (U/L)'].astype('str')
temp.obs['AST (U/L)'] = temp.obs['AST (U/L)'].astype('str')

In [97]:
#save updated rawdata
adata_savepath = '/domino/edv/id-td-virology/Zhiyuan/public/Gut_remap/processed_data/annotated.h5ad'
temp.write(adata_savepath)

